In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import csv

In [2]:
model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)  
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.to('cuda')

inputs = [
    "en: Whilst a lot of palaces were destroyed in the citadel in the Vietnam war its still very pleasurable and a lot to see the area is far to big to walk around, but possible if you start out early .Dont use those pedal bike taxis, its starts at 100,000 dong and ends up at a million .im not stupid with money , but i did get caught out by them, getting caught on the sympathy trip.tip.You can get some electric courtesy buses at the main entrance so ask about them i think but i saw them later on and there arnt too many about from what i saw .However enough belly acheing this site is well worth the trip one of the highlights of my trip to vietnam not so touristy, as Hoyan and after the city very quiet and peaceful.",
    ]
outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids.to('cuda'), max_length=110)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['vi: Trong khi rất nhiều cung điện đã bị phá huỷ ở thành cổ trong chiến tranh Việt Nam, nó vẫn rất thú vị và rất nhiều thứ để xem, khu vực này không phải là lớn để đi bộ xung quanh, nhưng có thể nếu bạn bắt đầu sớm.không sử dụng những chiếc taxi đạp xe, nó bắt đầu với 100.000 đồng và kết thúc với một triệu đồng. tôi không ngu ngốc với tiền, nhưng tôi đã bị bắt bởi chúng, bị bắt trên chuyến đi thông cảm.']


In [3]:
with open('reviews-after-VietAI-envit5-translation.csv', 'w', newline='') as csvfile2:
    fieldnames = ['en_text', 'positive_score', 'negative_score', 'neutral_score', 'vn_text']
    writer = csv.DictWriter(csvfile2, fieldnames=fieldnames)
    writer.writeheader()

    with open("data/reviews-after-cardiffnlp-twitter-roberta-base-sentiment-latest.csv", 'r') as csvfile1:
        csv_reader = csv.reader(csvfile1)
        next(csv_reader, None)
        for row in csv_reader:
            en_text = 'en: ' + row[0]
            positive_score = row[1]
            negative_score = row[2]
            neutral_score = row[3]
      
            inputs = [en_text]
            outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids.to('cuda'), max_length=110)
            vn_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

            writer.writerow({
                'en_text': en_text,
                'positive_score': positive_score,
                'negative_score': negative_score,
                'neutral_score': neutral_score,
                'vn_text' : vn_text,
                })